# Aula 09 - CARS - Exercícios

In [1]:
!pip install numpy==1.26.4

In [2]:
import pandas as pd
import numpy as np

### Importar base de dados

In [3]:
!wget https://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip ml-100k.zip

--2025-10-06 22:41:18--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.96.204
Connecting to files.grouplens.org (files.grouplens.org)|128.101.96.204|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  11.1MB/s    in 0.4s    

2025-10-06 22:41:19 (11.1 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.ba

In [4]:
rating_cols = ['user_id', 'item_id', 'rating', 'timestamp']
rating = pd.read_csv('./ml-100k/u.data', sep='\t', names=rating_cols)
rating.drop('timestamp', axis=1, inplace=True)
rating.head()

,user_id,item_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [5]:
#Load the Movies data
item_cols = ['item_id', 'title', 'release', 'video_release_date', 'IMDb_URL', 'unknown', 'Action',
             'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
             'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
item = pd.read_csv('./ml-100k/u.item', sep='|', names=item_cols, encoding='latin-1')
# Manteremos as colunas de gênero para usar como features
item = item.drop(columns=['video_release_date', 'IMDb_URL', 'unknown'])

item.head()

,item_id,title,release,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [6]:
#Load the User data
user = pd.read_csv('./ml-100k/u.user', sep='|', encoding='latin-1', header=None)
user.columns = ['user_id', 'age', 'gender', 'occupation', 'zipcode']
user.head()

,user_id,age,gender,occupation,zipcode
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


### Obs. Para o exercício abaixo, você poderá resolvê-lo usando a implementação própria contida neste notebook, ou então usar a biblioteca fastFM, cujo exemplo de uso está no notebook Aula09_Exemplos_fastFM.ipynb

***Exercício 01:*** Na aula, vimos a implementação e a aplicação do Factorization Machines (FM) com dois tipos de contexto: gêneros dos filmes e profissão do usuário. Verifique o desempenho do RMSE quando mais informações de contexto são incorporadas:
- Década de lançamento do filme: classifique os filmes por década: 60's, 70's, 80's, 90's, etc.
- Sexo do usuário: M ou F.
- Faixa etária: classifique os usuários por faixa etária, por exemplo: criança (até 12 anos), adolescente (13 a 17 anos), adulto (18 a 50 anos) e idoso (acima de 51 anos).
- CEP: utilizar a biblioteca https://www.pythonpool.com/uszipcode-python/ de acordo com o exemplo abaixo

### Obter informações do usuário a partir do CEP (https://www.pythonpool.com/uszipcode-python/)

In [7]:
!pip install fastFM
!pip install uszipcode==0.2.6 sqlalchemy==1.4.46 sqlalchemy-mate==1.4.28.4

In [8]:
from uszipcode import SearchEngine
from scipy.sparse import csr_matrix, hstack

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from fastFM import sgd

In [9]:
# engine = SearchEngine()
# zipcode = engine.by_zipcode(85711)

# print(zipcode.state)

### Consultando o estado

In [10]:
print(rating)

       user_id  item_id  rating
0          196      242       3
1          186      302       3
2           22      377       1
3          244       51       2
4          166      346       1
...        ...      ...     ...
99995      880      476       3
99996      716      204       5
99997      276     1090       1
99998       13      225       2
99999       12      203       3

[100000 rows x 3 columns]


In [11]:
# * Década de Lançamento ---
# Converte a data de lançamento para o formato de data e pega o ano
item['release_date'] = pd.to_datetime(item['release'], errors='coerce') # Use errors='coerce' to handle invalid date formats
item['release_year'] = item['release_date'].dt.year

# Remove rows where 'release_year' is null BEFORE calculating the decade
item.dropna(subset=['release_year'], inplace=True)

# Coluna 'decade'
# Now that release_year is not null, we can directly calculate the decade
item['decade'] = item['release_year'].apply(lambda x: int((x // 10) * 10))

item.drop(columns=['release_date', 'release_year'], inplace=True)


# * Faixa Etária
bins = [0, 12, 17, 50, 100]
labels = ['criança', 'adolescente', 'adulto', 'idoso']
user['age_group'] = pd.cut(user['age'], bins=bins, labels=labels, right=True)


# * Estado a partir do CEP
search = SearchEngine()

# Mapa de CEP -> Estado
unique_zipcodes = user['zipcode'].unique()
zip_to_state = {}
for zipcode in unique_zipcodes:
    try:
        # caso nao ache o cep
        result = search.by_zipcode(zipcode)
        if result:
            zip_to_state[zipcode] = result.state
        else:
            zip_to_state[zipcode] = 'unknown'
    except:
        zip_to_state[zipcode] = 'unknown'

user['state'] = user['zipcode'].map(zip_to_state)

# Juntando tudo em um único DataFrame
df = pd.merge(rating, user, on='user_id')
df = pd.merge(df, item, on='item_id')

df.dropna(inplace=True)

print("DataFrame final com features de contexto:")
df.head()

DataFrame final com features de contexto:


,user_id,item_id,rating,age,gender,occupation,zipcode,age_group,state,title,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,decade
0,196,242,3,49,M,writer,55105,adulto,MN,Kolya (1996),...,0,0,0,0,0,0,0,0,0,1990
1,186,302,3,39,F,executive,00000,adulto,unknown,L.A. Confidential (1997),...,1,0,0,1,0,0,1,0,0,1990
2,22,377,1,25,M,writer,40206,adulto,KY,Heavyweights (1994),...,0,0,0,0,0,0,0,0,0,1990
3,244,51,2,28,M,technician,80525,adulto,CO,Legends of the Fall (1994),...,0,0,0,0,1,0,0,1,1,1990
4,166,346,1,47,M,educator,55113,adulto,MN,Jackie Brown (1997),...,0,0,0,0,0,0,0,0,0,1990


In [12]:
# Features Categóricas de Contexto + IDs
categorical_features = ['user_id', 'item_id', 'gender', 'occupation', 'age_group', 'decade']

# OneHotEncoder
for col in categorical_features:
    df[col] = df[col].astype(str)


encoder = OneHotEncoder(handle_unknown='ignore')
X_categorical = encoder.fit_transform(df[categorical_features])


# Features de Gênero
genre_cols = ['Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
              'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
X_genres = csr_matrix(df[genre_cols].values)

X = hstack([X_categorical, X_genres])

y = df['rating'].values

print(f"Shape of the final feature matrix X: {X.shape}")
print(f"Shape of the target vector y: {y.shape}")



Shape of the final feature matrix X: (99991, 2677)
Shape of the target vector y: (99991,)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelo FastFM ---
fm = sgd.FMRegression(
    n_iter = 5000000,  # iterações do gradiente descendente
    rank = 20,       # dimensionalidade fatores latentes (k)
    step_size = 0.001 # tx de aprendizado (learning rate)
)

print("* Trainando o modelo")
fm.fit(X_train, y_train)
print("** Treinamento concluído.")

# Erro quadradtico médio
y_pred = fm.predict(X_test)

# Calcula o erro quadrático médio e, em seguida, a raiz (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"\nO desempenho do modelo")
print(f"RMSE: {rmse:.4f}")

* Trainando o modelo
** Treinamento concluído.

O desempenho do modelo
RMSE: 0.9280
